**Prototyping metrics-driven report**

Create a metrics-driven report of trends and topics of interest, and how they shift over time

In [1]:
from pandas import read_csv
import pandas as pd

In [ ]:
''' work with engagement data '''
from google.colab import files
files.upload()
import os
print(os.listdir())

In [ ]:
''' perform basic data exploration '''
df = pd.read_csv('engagements.csv')
df.head()[:5]
df.info()
display(df.describe())

In [ ]:
''' Handle any missing values, convert data types if necessary,
and extract relevant information (like date/time components)
'''
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')
df['date'] = df['timestamp'].dt.date
df['media_caption'].fillna('', inplace=True)
df['comment_text'].fillna('', inplace=True)
display(df.head())
display(df.info())

Engagement trends analysis

In [ ]:
'''  Analyze engagement trends
how engagement metrics (likes, comments, etc.) change over time, this could be
time series analysis to identify daily, weekly, or monthly patterns

Group the DataFrame by date, count the entries, reset the index,
and sort by date to prepare for time series analysis.
'''
daily_engagement = df.groupby('date').size().reset_index(name='engagement_count')
daily_engagement['date'] = pd.to_datetime(daily_engagement['date'])
daily_engagement = daily_engagement.sort_values(by='date')
display(daily_engagement.head())

In [ ]:
'''
the weekly and monthly engagement by resampling the daily engagement data and
then display the first few rows of the daily, weekly, and monthly engagement
dataframes to verify the results
'''
weekly_engagement = daily_engagement.resample('W', on='date')[
    'engagement_count'].sum().reset_index(name='weekly_engagement_count')
monthly_engagement = daily_engagement.resample('M', on='date')[
    'engagement_count'].sum().reset_index(name='monthly_engagement_count')

print("Daily Engagement:")
display(daily_engagement.head())
print("\nWeekly Engagement:")
display(weekly_engagement.head())
print("\nMonthly Engagement:")
display(monthly_engagement.head())

Identify topics of interest

In [ ]:
'''
If the data includes post content or tags, perform text analysis
(like topic modeling or keyword extraction) to identify popular themes

Combine the text columns, clean the text by converting to lowercase,
removing punctuation, and removing stop words, and then calculate word frequencies.
'''

import string
from nltk.corpus import stopwords
import nltk

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

# 1. Combine the media_caption and comment_text columns
df['combined_text'] = df['media_caption'] + ' ' + df['comment_text']

# 2. Perform basic text cleaning
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

df['cleaned_text'] = df['combined_text'].apply(clean_text)

# Remove stop words
stop_words = set(stopwords.words('english'))
df['cleaned_text'] = df['cleaned_text'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# 3. Use a simple word frequency count to identify important terms
word_counts = {}
for text in df['cleaned_text']:
    for word in text.split():
        word_counts[word] = word_counts.get(word, 0) + 1

# Sort words by frequency
sorted_word_counts = sorted(
    word_counts.items(), key=lambda item: item[1], reverse=True)

# Display the top 50 most frequent words
print("Top 50 most frequent words:")
for word, count in sorted_word_counts[:50]:
    print(f"{word}: {count}")

# Display the first few rows with the new columns
display(
  df[['media_caption', 'comment_text', 'combined_text', 'cleaned_text']].head())

Topic Trends analysis

In [ ]:
''' Examine how the popularity of identified topics shifts over time

Create a combined text column, define keywords for each topic, iterate through
rows to create boolean topic columns, group by date to calculate daily popularity,
convert date to datetime, and sort.
'''
df['combined_text'] = df['media_caption'] + ' ' + df['comment_text']

topic_keywords = {
    'brand_product': ['tree hut', 'scrub', 'butter', 'wash',
                      'scent', 'skin', 'product', 'smell', 'body'],
    'marketing_promotions': ['giveaway', 'contest', 'win', 'enter',
                             'winners', 'tag', 'share', 'follow',
                             'chance', 'promo', 'discount'],
    'user_sentiment': ['love', 'good', 'great', 'amazing', 'help',
                       'favorite', 'want', 'need', 'obsessed', 'repurchase']
}

for topic, keywords in topic_keywords.items():
    df[topic] = df['combined_text'].apply(
        lambda x: any(keyword in x.lower() for keyword in keywords))

daily_topic_popularity = df.groupby('date')[
    list(topic_keywords.keys())].mean().reset_index()
daily_topic_popularity['date'] = pd.to_datetime(daily_topic_popularity['date'])
daily_topic_popularity = daily_topic_popularity.sort_values(by='date')

display(daily_topic_popularity.head())

In [ ]:
''' Resample the daily topic popularity data to weekly and monthly
frequencies by summing the popularity scores
'''
weekly_topic_popularity = daily_topic_popularity.resample(
    'W', on='date')[list(topic_keywords.keys())].sum().reset_index()
monthly_topic_popularity = daily_topic_popularity.resample(
    'M', on='date')[list(topic_keywords.keys())].sum().reset_index()

print("Weekly Topic Popularity:")
display(weekly_topic_popularity.head())
print("\nMonthly Topic Popularity:")
display(monthly_topic_popularity.head())

Correlate topics and engagement

In [ ]:
'''
Correlate topics and engagement

Merge the daily engagement and daily topic popularity dataframes on the 'date'
column, calculate the correlation matrix, and print
'''
daily_trends = pd.merge(daily_engagement, daily_topic_popularity, on='date')
correlation_matrix = daily_trends.corr(numeric_only=True)
print("Correlation matrix between engagement and topic popularity:")
display(correlation_matrix[['engagement_count']])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting daily engagement trends
plt.figure(figsize=(15, 6))
plt.plot(daily_engagement['date'], daily_engagement['engagement_count'])
plt.title('Daily Engagement Trends')
plt.xlabel('Date')
plt.ylabel('Engagement Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plotting weekly engagement trends
plt.figure(figsize=(15, 6))
plt.plot(weekly_engagement['date'], weekly_engagement['weekly_engagement_count'])
plt.title('Weekly Engagement Trends')
plt.xlabel('Date')
plt.ylabel('Weekly Engagement Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# Plotting monthly engagement trends
plt.figure(figsize=(15, 6))
plt.plot(monthly_engagement['date'], monthly_engagement['monthly_engagement_count'])
plt.title('Monthly Engagement Trends')
plt.xlabel('Date')
plt.ylabel('Monthly Engagement Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plotting daily topic popularity trends
plt.figure(figsize=(15, 6))
for topic in topic_keywords.keys():
    plt.plot(daily_topic_popularity['date'], daily_topic_popularity[topic], label=topic)
plt.title('Daily Topic Popularity Trends')
plt.xlabel('Date')
plt.ylabel('Popularity (Mean Engagement)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Plotting weekly topic popularity trends
plt.figure(figsize=(15, 6))
for topic in topic_keywords.keys():
    plt.plot(weekly_topic_popularity['date'], weekly_topic_popularity[topic], label=topic)
plt.title('Weekly Topic Popularity Trends')
plt.xlabel('Date')
plt.ylabel('Popularity (Sum of Daily Means)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Plotting monthly topic popularity trends
plt.figure(figsize=(15, 6))
for topic in topic_keywords.keys():
    plt.plot(monthly_topic_popularity['date'], monthly_topic_popularity[topic], label=topic)
plt.title('Monthly Topic Popularity Trends')
plt.xlabel('Date')
plt.ylabel('Popularity (Sum of Daily Means)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Generating heatmap of correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix: Engagement vs. Topic Popularity')
plt.tight_layout()
plt.show()

Data Analysis Key Findings
- Daily engagement shows significant fluctuations,
- Brand/Product" and "User Sentiment" topic popularity generally exhibit similar daily and weekly patterns.
- There is a moderate positive correlation (0.39) between "user_sentiment" popularity and engagement count.
- There is a moderate negative correlation (-0.40) between "brand_product" popularity and engagement count.

Insights or Next Steps
- Focus on content that emphasizes "User Sentiment" (opinions, positive experiences, desired products) as it is most positively correlated with engagement.
- Investigate the unexpected negative correlation with "Brand/Product" content to understand if the method of presentation (e.g., overly promotional vs. integrated into user stories) impacts engagement.
